# Tutorial 8: Graph Creation Benchmark

## Introduction

In enterprise systems, the entire analytical logic and data flow can be so complex that
the resulting computational DAG (directed acyclic graph) is extremely large, for example,
with hundreds of millions of nodes. It is extremely time-consuming to create such large
computational DAGs. Often the time it takes to create these large DAGs is much longer than
the time to execute them. Therefore, it is crucial for a graph computing solution to be
able to create large DAGs quickly, otherwise it won't be a viable solution for
complex enterprise use cases.

Julius Graph Engine features a low-code domain specific language, RuleDSL, which is
designed specifically for creating DAGs. RuleDSL not only makes it easy for developers to
write complex business logic, but also enables fast and efficient creations of large
computational DAGs.

In this study, we compare the performance of DAG creation between
[Julius](http://juliustech.co) and a few other well-known graph computing packages,
such as [Dask](http://dask.org), [Dagger.jl](https://github.com/JuliaParallel/Dagger.jl) and
[Tensorflow](http://tensorflow.org). The comparison is performed on a single computer without
distribution, because even though all solutions supports parallel execution of computational
DAGs, only Julius supports parallelism for DAG creation.

## Benchmark Setup

The problem we used for benchmarking is to compute the value of a simple Fibonancci like
sequence of:

$y_n = .7  y_{n-1} + .3 y_{n-2}$

where any $y_i$ is a vector of length 10. The initial terms $y_0$ and $y_1$ are
random vectors of length 10.

The methodology of the benchmarking is straightforward: we simply create and run the
computational DAGs using different graph computing solutions, then record their wall clock
time. Python time is measured by `%time`, and Julia time is measured by `@time`. Since the
numerical computation of the sequence is trivial, the time recorded is almost 100%
on the creation of computational DAG.

We want to emphasize that this benchmarking exercise is only for the speed of
DAG creation. We are not testing any other features in these software packages. However,
given the graph creation is often the most time-consuming step for running large systems
as DAGs, it is of great practical interests to understand its performance characteristics.

The source code for the benchmarking is listed in the appendix.

## Results

The hardware for running the benchmark is a single laptop with a 6-core intel i-7 CPU and
64 GB of memory. The term $n$ is chosen to be 20, 1,000, 10,000, 100,000, 500,000. The
following is the results of the benchmark, all timing numbers are reported in seconds.
Fail indicates that the run does not complete within 12 hours, which is too long for any
practical usage.

| n | Dask | Dagger.jl | Tensorflow | Julius |
| :---: | :---: | :---: | :---: | :---: |
| 20 | 0.004 | 0.08 | 0.001 | 0.0007 |
| 1,000 | 1.35 | fail | 2.19 | 0.026 |
| 10,000 | 17 | fail | 23 | 0.3 |
| 100,000 | 3029 | fail | 317 | 5 |
| 500,000 | fail | fail | 1554 | 64 |

It seems the current version of Dagger.jl (v0.14.3) has a bug that prevents the benchmark
from completing for even small n, the issue has been reported to Dagger.jl developers.
Another way to look at the results is to estimate the maximum size of a DAG that can be
supported on a single computer, assuming 6 hours is the practical upper limit for DAG
creation:

| Solutions | Dask | Dagger.jl | Tensorflow | Julius |
| :---: | :---: | :---: | :---:| :---: |
|Max Graph Size | < 500K | < 1K | 10 MM | 300 MM |


## Conclusion

The graph creation benchmark clearly showed that Julius has a huge speed advantage for
creating large graphs, thanks to its RuleDSL.

Julius can create DAGs with hundreds of millions of nodes using a single computer, which
is sufficient to cover even the most complex enterprise use cases. In addition, Julius'
graph construction can be easily parallelized thanks to the simple syntax of RuleDSL,
thus extending Julius' upper limit to billions of nodes, if such needs ever arise in
practice.

In comparison, Dask, Dagger.jl and Tensorflow do not support parallelism for DAG creation.
Therefore, the DAG creation is much more likely to become a bottleneck for enterprise
problems using these solutions.


## Appendix: Source Code

All the code below are directly runnable once the dependent packages are installed.

The cleanest way to implement this sequence is via recursion. However, Dask,
Dagger.jl and Tensorflow does not yet support recursive functions, therefore we have
to write an explicit loop in their implementation. Julius' RuleDSL does support recursive
definitions, which is used below for defining the sequence in Julius implementation.

#### **Dask implementation**

```python
import dask
import numpy as np

@dask.delayed
def fib0(n) :
        return np.random.rand(10)

@dask.delayed
def wsum(a, b) :
    return  (.3*a + .7*b)

%%time
# %%time is a magic command, only works in Jupyter notebook

f0 = fib0(0)
f1 = fib0(1)

for i in range(0, 10000) :
    f2 = wsum(f0, f1)
    f0, f1 = f1, f2
```

#### **Dagger.jl implementation**

```julia

using Dagger
fibsum(a, b)=.3 .* a .+ .7 .* b

f0 = Dagger.@spawn rand(10)
f1 = Dagger.@spawn rand(10)
f2 = 0 # final result will be held here

@time for i in 1:20
    f2 = Dagger.@spawn fibsum(f0, f1)
    f0, f1 = f1, f2
end

```

#### **Tensorflow implementation**

```python
import tensorflow as tf
import numpy as np

a = tf.Variable(np.random.rand(10))
b = tf.Variable(np.random.rand(10))

@tf.function
def wsum(a, b) :
    return a*.3 + b*.7

# have to wrap the top level call by @tf.function, otherwise
# TF2 does not create the computational graph
@tf.function
def fib(n, f0, f1) :
    # not using tf.range() because Tensorflow automatically
    # optimizes tf.range() into a single loop node instead of
    # creating n number of nodes in the graph
    for i in range(0, n) :
        f2 = wsum(f0, f1)
        f0, f1 = f1, f2
    return f2

%%time
# %%time is magic command, only works in Jupyter notebook

fib(1000, a, b)
```

#### **Julius implementation**

To learn more about JuliusDSL and its implementation, please refer to the quickstart
tutorial. The `ApplyFn` is a convenient Atom that allows arbitrary Julia function to
be used in `RuleDSL`, please refer to the mapreduce tutorial for the details on `ApplyFn`.

```julia
using GraphEngine: RuleDSL, GraphVM
using DataScience: ApplyFn

# ApplyFn is a generic Atom allows arbitrary julia function to be called
RuleDSL.@addrules seq begin
    fib(n::Int) = RuleDSL.Alias(fib(n, Val(n <= 1)))
    fib(n::Int, isend::Val{false}) =
        ApplyFn[(a, b)-> (.7 .* a .+ .3 .* b)](
            fib(n - 1, Val(n <= 2)), fib(n - 2, Val(n <= 3))
        )
    fib(n::Int, isend::Val{true}) = ApplyFn[()->rand(10)]()
end

ref = RuleDSL.@ref seq.fib(10000)
gs = GraphVM.createlocalgraph(RuleDSL.Config(), RuleDSL.GenericData());
@time GraphVM.calcfwd!(gs, Set([ref]));

```

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*